In [1]:
import osmnx as ox, networkx as nx, pandas as pd, geopandas as gpd, time, matplotlib.pyplot as plt, math, ast, re
import matplotlib.cm as cm
from matplotlib.collections import PatchCollection
from descartes import PolygonPatch
from shapely.geometry import Point, Polygon, MultiPolygon
import statsmodels.api as sm, numpy as np
from geopy.distance import great_circle
from shapely.geometry import Polygon
import ast
%matplotlib inline
ox.config(use_cache=True, log_console=True)

In [2]:
df = pd.read_csv('data/sf-nhoods.csv')

In [6]:
df['intersection_density_km']

0      50.627
1     126.015
2     101.500
3     115.232
4      90.274
5      48.109
6      93.391
7      73.048
8      66.797
9     109.849
10     75.946
11     72.760
12     62.161
13     51.855
14     99.975
15     90.866
16     81.714
17     79.400
18     61.278
19     76.646
20    102.932
21     63.566
22     52.799
23     74.978
24     56.861
25     64.169
26     78.643
27     96.791
28     22.984
29     67.425
30     61.518
31     62.003
32     99.206
33    102.753
Name: intersection_density_km, dtype: float64